In [ ]:
import os
import shutil
import sys
from google.colab import drive

#  Mount Google Drive
drive.mount('/content/drive')

#  Restore Model Code
if not os.path.exists('/content/models'):
    print("Restoring Code...")
    !git clone https://github.com/Gabrysse/MLDL2024_project1.git temp_repo
    shutil.copytree('temp_repo/models', '/content/models')
    shutil.rmtree('temp_repo')

#  Add models to system path
sys.path.append('/content/models')

#  Restore Dataset (GTA5 + Cityscapes)
if not os.path.exists('/content/dataset/project_data/gta5'):
    print("Restoring Dataset...")
    zip_path_1 = '/content/drive/MyDrive/semseg/project_data.zip'
    zip_path_2 = '/content/drive/MyDrive/project_data.zip'

    if os.path.exists(zip_path_1):
        shutil.unpack_archive(zip_path_1, '/content/dataset')
        print("Dataset extracted.")
    elif os.path.exists(zip_path_2):
        shutil.unpack_archive(zip_path_2, '/content/dataset')
        print("Dataset extracted.")
    else:
        print("Error: project_data.zip not found.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import os
import random
import sys

# --- CONFIGURATION ---
CHECKPOINT_NAME = 'bisenet_extension_final.pth'
# Pointing to your BEST model (Thresholded - 24.83%)
PRETRAINED_PATH = '/content/drive/MyDrive/semseg/bisenet_dacs_thresholded.pth'
SAVE_PATH = f'/content/drive/MyDrive/semseg/{CHECKPOINT_NAME}'

EPOCHS = 15
BATCH_SIZE = 4
LR = 1e-3    # Low LR for fine-tuning
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
GTA_PATH = '/content/dataset/project_data/gta5'
CITYSCAPES_PATH = '/content/dataset/project_data/cityscapes'

# --- IMPORTS ---
if os.path.exists('/content/MLDL2024_project1'):
    sys.path.append('/content/MLDL2024_project1')
try:
    from models.bisenet.build_bisenet import BiSeNet
except ImportError:
    print("Error: BiSeNet not found.")

# --- 1. DICE LOSS IMPLEMENTATION ---
class DiceLoss(nn.Module):
    def __init__(self, num_classes=19, smooth=1.0, ignore_index=255):
        super(DiceLoss, self).__init__()
        self.num_classes = num_classes
        self.smooth = smooth
        self.ignore_index = ignore_index

    def forward(self, pred, target):
        pred = F.softmax(pred, dim=1)
        target_one_hot = F.one_hot(target.clamp(0, self.num_classes-1), num_classes=self.num_classes)
        target_one_hot = target_one_hot.permute(0, 3, 1, 2).float()

        mask = (target != self.ignore_index).unsqueeze(1).float()
        pred = pred * mask
        target_one_hot = target_one_hot * mask

        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))

        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return 1.0 - dice.mean()

# --- 2. DATASET ---
class GTA5_City_Dataset(Dataset):
    def __init__(self, gta_root, city_root):
        self.gta_images_dir = os.path.join(gta_root, 'images')
        self.gta_masks_dir = os.path.join(gta_root, 'labels')
        self.gta_images = sorted(os.listdir(self.gta_images_dir))
        self.city_images_dir = os.path.join(city_root, 'leftImg8bit', 'train')
        self.city_images = []
        if os.path.exists(self.city_images_dir):
            for city in os.listdir(self.city_images_dir):
                c_path = os.path.join(self.city_images_dir, city)
                if os.path.isdir(c_path):
                    for f in os.listdir(c_path):
                        if f.endswith('_leftImg8bit.png'):
                            self.city_images.append(os.path.join(c_path, f))

        self.normalize = transforms.Compose([
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.id_mapping = {
            7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
            19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
            26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18
        }
        self.to_tensor = transforms.ToTensor()

    def __len__(self): return len(self.gta_images)
    def __getitem__(self, idx):
        gta_path = os.path.join(self.gta_images_dir, self.gta_images[idx])
        mask_path = os.path.join(self.gta_masks_dir, self.gta_images[idx])
        gta_img = Image.open(gta_path).convert('RGB').resize((1280, 720), Image.BILINEAR)
        gta_mask = Image.open(mask_path).resize((1280, 720), Image.NEAREST)

        rand_idx = random.randint(0, len(self.city_images) - 1)
        city_img = Image.open(self.city_images[rand_idx]).convert('RGB').resize((1280, 720), Image.BILINEAR)

        gta_t = self.normalize(self.to_tensor(gta_img))
        city_t = self.normalize(self.to_tensor(city_img))

        mask_np = np.array(gta_mask)
        gta_lbl = np.full(mask_np.shape, 255, dtype=np.uint8)
        for k, v in self.id_mapping.items(): gta_lbl[mask_np == k] = v

        return gta_t, torch.from_numpy(gta_lbl).long(), city_t

# --- 3. TRAINING LOOP ---
print("Starting Step 5 Extension: Hybrid Loss Fine-Tuning...")

model = BiSeNet(num_classes=19, context_path='resnet18').to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

# THE EXTENSION: Using two losses together
ce_criterion = nn.CrossEntropyLoss(ignore_index=255)
dice_criterion = DiceLoss(num_classes=19, ignore_index=255)

if os.path.exists(PRETRAINED_PATH):
    print(f"Loading Best Model: {PRETRAINED_PATH}")
    checkpoint = torch.load(PRETRAINED_PATH, map_location=DEVICE)

    # Robust Loading logic
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
        print("Model loaded (state_dict).")
    elif isinstance(checkpoint, dict) and 'student' in checkpoint:
         model.load_state_dict(checkpoint['student'])
         print("Model loaded (student key).")
    else:
        model.load_state_dict(checkpoint)
        print("Model loaded (direct).")
else:
    print("Error: Best model not found. Cannot fine-tune.")
    sys.exit()

dataset = GTA5_City_Dataset(GTA_PATH, CITYSCAPES_PATH)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

model.train()
print(f"Training for {EPOCHS} epochs...")

for epoch in range(EPOCHS):
    for i, (gta_img, gta_lbl, city_img) in enumerate(loader):
        gta_img, gta_lbl, city_img = gta_img.to(DEVICE), gta_lbl.to(DEVICE), city_img.to(DEVICE)

        optimizer.zero_grad()

        # 1. Pseudo-Labeling (Student Self-Training)
        with torch.no_grad():
            city_out = model(city_img)
            if isinstance(city_out, tuple): city_out = city_out[0]
            city_pseudo_lbl = torch.argmax(city_out, dim=1)

        # 2. Mix (ClassMix)
        batch_size = gta_img.shape[0]
        mixed_img = city_img.clone()
        mixed_lbl = city_pseudo_lbl.clone()
        for b in range(batch_size):
            classes = torch.unique(gta_lbl[b])
            classes = classes[classes != 255]
            if len(classes) > 0:
                perm = torch.randperm(len(classes))
                sel = classes[perm[:(len(classes)+1)//2]]
                mask = torch.zeros_like(gta_lbl[b]).bool()
                for c in sel: mask = mask | (gta_lbl[b] == c)
                mixed_img[b, :, mask] = gta_img[b, :, mask]
                mixed_lbl[b, mask] = gta_lbl[b, mask]

        # 3. Forward
        out = model(mixed_img)

        # 4. HYBRID LOSS CALCULATION
        loss_ce = ce_criterion(out[0], mixed_lbl)
        loss_dice = dice_criterion(out[0], mixed_lbl)

        # Weighted combination
        total_loss = loss_ce + loss_dice
        total_loss += 0.1 * ce_criterion(out[1], mixed_lbl) + 0.1 * ce_criterion(out[2], mixed_lbl)

        total_loss.backward()
        optimizer.step()

        if i % 50 == 0:
            print(f"Ep {epoch+1} | It {i} | Loss: {total_loss.item():.4f}")

    torch.save({'model_state_dict': model.state_dict(), 'epoch': epoch}, SAVE_PATH)
    print(f"Epoch {epoch+1} Saved.")

Starting Step 5 Extension: Hybrid Loss Fine-Tuning...
Loading Best Model: /content/drive/MyDrive/semseg/bisenet_dacs_thresholded.pth
Model loaded (state_dict).
Training for 15 epochs...
Ep 1 | It 0 | Loss: 1.1188
Ep 1 | It 50 | Loss: 1.1375
Ep 1 | It 100 | Loss: 1.4799
Ep 1 | It 150 | Loss: 1.0322
Ep 1 | It 200 | Loss: 1.0758
Ep 1 | It 250 | Loss: 1.1858
Ep 1 | It 300 | Loss: 1.0374
Ep 1 | It 350 | Loss: 1.1032
Ep 1 | It 400 | Loss: 0.9482
Ep 1 | It 450 | Loss: 1.0889
Ep 1 | It 500 | Loss: 1.1200
Ep 1 | It 550 | Loss: 0.9256
Ep 1 | It 600 | Loss: 0.9499
Epoch 1 Saved.
Ep 2 | It 0 | Loss: 0.9258
Ep 2 | It 50 | Loss: 0.9396
Ep 2 | It 100 | Loss: 0.9202
Ep 2 | It 150 | Loss: 0.9819
Ep 2 | It 200 | Loss: 1.0533
Ep 2 | It 250 | Loss: 0.8760
Ep 2 | It 300 | Loss: 0.9970
Ep 2 | It 350 | Loss: 0.9672
Ep 2 | It 400 | Loss: 0.9664
Ep 2 | It 450 | Loss: 0.9077
Ep 2 | It 500 | Loss: 0.9104
Ep 2 | It 550 | Loss: 1.0650
Ep 2 | It 600 | Loss: 1.0156
Epoch 2 Saved.
Ep 3 | It 0 | Loss: 0.9137
Ep 3 | It

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import os
import random
import sys

# --- CONFIGURATION ---
CHECKPOINT_NAME = 'bisenet_extension_final.pth'
# Your baseline best model
PRETRAINED_PATH = '/content/drive/MyDrive/semseg/bisenet_dacs_thresholded.pth'
# Where the new training is saved
SAVE_PATH = f'/content/drive/MyDrive/semseg/{CHECKPOINT_NAME}'

EPOCHS = 15
BATCH_SIZE = 4       # OOM Safe
ACCUM_STEPS = 2      # Gradient Accumulation
LR = 1e-3
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
GTA_PATH = '/content/dataset/project_data/gta5'
CITYSCAPES_PATH = '/content/dataset/project_data/cityscapes'

# --- IMPORTS ---
if os.path.exists('/content/MLDL2024_project1'):
    sys.path.append('/content/MLDL2024_project1')
try:
    from models.bisenet.build_bisenet import BiSeNet
except ImportError:
    print("Error: BiSeNet not found.")

# --- DICE LOSS ---
class DiceLoss(nn.Module):
    def __init__(self, num_classes=19, smooth=1.0, ignore_index=255):
        super(DiceLoss, self).__init__()
        self.num_classes = num_classes
        self.smooth = smooth
        self.ignore_index = ignore_index

    def forward(self, pred, target):
        pred = F.softmax(pred, dim=1)
        target_one_hot = F.one_hot(target.clamp(0, self.num_classes-1), num_classes=self.num_classes)
        target_one_hot = target_one_hot.permute(0, 3, 1, 2).float()

        mask = (target != self.ignore_index).unsqueeze(1).float()
        pred = pred * mask
        target_one_hot = target_one_hot * mask

        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))

        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return 1.0 - dice.mean()

# --- DATASET ---
class GTA5_City_Dataset(Dataset):
    def __init__(self, gta_root, city_root):
        self.gta_images_dir = os.path.join(gta_root, 'images')
        self.gta_masks_dir = os.path.join(gta_root, 'labels')
        self.gta_images = sorted(os.listdir(self.gta_images_dir))
        self.city_images_dir = os.path.join(city_root, 'leftImg8bit', 'train')
        self.city_images = []
        if os.path.exists(self.city_images_dir):
            for city in os.listdir(self.city_images_dir):
                c_path = os.path.join(self.city_images_dir, city)
                if os.path.isdir(c_path):
                    for f in os.listdir(c_path):
                        if f.endswith('_leftImg8bit.png'):
                            self.city_images.append(os.path.join(c_path, f))

        self.normalize = transforms.Compose([
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.id_mapping = {
            7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
            19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
            26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18
        }
        self.to_tensor = transforms.ToTensor()

    def __len__(self): return len(self.gta_images)
    def __getitem__(self, idx):
        gta_path = os.path.join(self.gta_images_dir, self.gta_images[idx])
        mask_path = os.path.join(self.gta_masks_dir, self.gta_images[idx])
        gta_img = Image.open(gta_path).convert('RGB').resize((1280, 720), Image.BILINEAR)
        gta_mask = Image.open(mask_path).resize((1280, 720), Image.NEAREST)

        rand_idx = random.randint(0, len(self.city_images) - 1)
        city_img = Image.open(self.city_images[rand_idx]).convert('RGB').resize((1280, 720), Image.BILINEAR)

        gta_t = self.normalize(self.to_tensor(gta_img))
        city_t = self.normalize(self.to_tensor(city_img))

        mask_np = np.array(gta_mask)
        gta_lbl = np.full(mask_np.shape, 255, dtype=np.uint8)
        for k, v in self.id_mapping.items(): gta_lbl[mask_np == k] = v

        return gta_t, torch.from_numpy(gta_lbl).long(), city_t

# --- TRAINING LOOP ---
print("Starting Step 5 Extension: Hybrid Loss (Resumable)...")
torch.cuda.empty_cache()

model = BiSeNet(num_classes=19, context_path='resnet18').to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

ce_criterion = nn.CrossEntropyLoss(ignore_index=255)
dice_criterion = DiceLoss(num_classes=19, ignore_index=255)

start_epoch = 0

# --- SMART RESUME LOGIC ---
if os.path.exists(SAVE_PATH):
    print(f"Found interrupted training: {SAVE_PATH}")
    print("Resuming from latest epoch...")
    checkpoint = torch.load(SAVE_PATH, map_location=DEVICE)

    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
        start_epoch = checkpoint.get('epoch', 0) + 1
    else:
        # Fallback if weird format
        model.load_state_dict(checkpoint)

elif os.path.exists(PRETRAINED_PATH):
    print(f"Starting Fresh Extension from Best Model: {PRETRAINED_PATH}")
    checkpoint = torch.load(PRETRAINED_PATH, map_location=DEVICE)
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    elif isinstance(checkpoint, dict) and 'student' in checkpoint:
         model.load_state_dict(checkpoint['student'])
    else:
        model.load_state_dict(checkpoint)
else:
    print("Error: No checkpoint found.")
    sys.exit()

dataset = GTA5_City_Dataset(GTA_PATH, CITYSCAPES_PATH)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)

model.train()
optimizer.zero_grad()

print(f"Training from Epoch {start_epoch} to {EPOCHS}...")

for epoch in range(start_epoch, EPOCHS):
    for i, (gta_img, gta_lbl, city_img) in enumerate(loader):
        gta_img, gta_lbl, city_img = gta_img.to(DEVICE), gta_lbl.to(DEVICE), city_img.to(DEVICE)

        # 1. Pseudo-Labeling
        with torch.no_grad():
            city_out = model(city_img)
            if isinstance(city_out, tuple): city_out = city_out[0]
            city_pseudo_lbl = torch.argmax(city_out, dim=1)

        # 2. Mix
        batch_size = gta_img.shape[0]
        mixed_img = city_img.clone()
        mixed_lbl = city_pseudo_lbl.clone()
        for b in range(batch_size):
            classes = torch.unique(gta_lbl[b])
            classes = classes[classes != 255]
            if len(classes) > 0:
                perm = torch.randperm(len(classes))
                sel = classes[perm[:(len(classes)+1)//2]]
                mask = torch.zeros_like(gta_lbl[b]).bool()
                for c in sel: mask = mask | (gta_lbl[b] == c)
                mixed_img[b, :, mask] = gta_img[b, :, mask]
                mixed_lbl[b, mask] = gta_lbl[b, mask]

        # 3. Forward
        out = model(mixed_img)

        # 4. Hybrid Loss
        loss_ce = ce_criterion(out[0], mixed_lbl)
        loss_dice = dice_criterion(out[0], mixed_lbl)

        total_loss = loss_ce + loss_dice
        total_loss += 0.1 * ce_criterion(out[1], mixed_lbl) + 0.1 * ce_criterion(out[2], mixed_lbl)

        # Gradient Accumulation
        total_loss = total_loss / ACCUM_STEPS
        total_loss.backward()

        if (i + 1) % ACCUM_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()

        if i % 100 == 0:
            print(f"Ep {epoch+1} | It {i} | Loss: {total_loss.item() * ACCUM_STEPS:.4f}")

    # Save at end of epoch
    torch.save({'model_state_dict': model.state_dict(), 'epoch': epoch}, SAVE_PATH)
    print(f"Epoch {epoch+1} Saved.")

Starting Step 5 Extension: Hybrid Loss (Resumable)...
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 200MB/s]


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


100%|██████████| 171M/171M [00:01<00:00, 127MB/s]


Found interrupted training: /content/drive/MyDrive/semseg/bisenet_extension_final.pth
Resuming from latest epoch...
Training from Epoch 2 to 15...
Ep 3 | It 0 | Loss: 0.9271
Ep 3 | It 100 | Loss: 0.9619
Ep 3 | It 200 | Loss: 0.8656
Ep 3 | It 300 | Loss: 0.8507
Ep 3 | It 400 | Loss: 0.8059
Ep 3 | It 500 | Loss: 0.8762
Ep 3 | It 600 | Loss: 0.9383
Epoch 3 Saved.
Ep 4 | It 0 | Loss: 0.8067
Ep 4 | It 100 | Loss: 0.7464
Ep 4 | It 200 | Loss: 0.9241
Ep 4 | It 300 | Loss: 0.7136
Ep 4 | It 400 | Loss: 0.7868
Ep 4 | It 500 | Loss: 0.7510
Ep 4 | It 600 | Loss: 0.6972
Epoch 4 Saved.
Ep 5 | It 0 | Loss: 0.7886
Ep 5 | It 100 | Loss: 0.7279
Ep 5 | It 200 | Loss: 0.7354
Ep 5 | It 300 | Loss: 0.8419
Ep 5 | It 400 | Loss: 0.7557
Ep 5 | It 500 | Loss: 0.6576
Ep 5 | It 600 | Loss: 0.7696
Epoch 5 Saved.
Ep 6 | It 0 | Loss: 0.7435
Ep 6 | It 100 | Loss: 0.5725
Ep 6 | It 200 | Loss: 0.7646
Ep 6 | It 300 | Loss: 0.7464
Ep 6 | It 400 | Loss: 0.6309
Ep 6 | It 500 | Loss: 0.9299
Ep 6 | It 600 | Loss: 0.7633
Epoc

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import os
import random
import sys

# --- CONFIGURATION ---
# SAVE AS A NEW FILE so we don't overwrite the 26.68% model if this gets worse
SAVE_NAME = 'bisenet_extension_30ep.pth'
# Load the model you just finished (26.68%)
PRETRAINED_PATH = '/content/drive/MyDrive/semseg/bisenet_extension_final.pth'
SAVE_PATH = f'/content/drive/MyDrive/semseg/{SAVE_NAME}'

TOTAL_EPOCHS = 30    # We are going from Epoch 15 -> 30
BATCH_SIZE = 4       # Keep OOM Safe settings
ACCUM_STEPS = 2
LR = 1e-3
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
GTA_PATH = '/content/dataset/project_data/gta5'
CITYSCAPES_PATH = '/content/dataset/project_data/cityscapes'

# --- IMPORTS ---
if os.path.exists('/content/MLDL2024_project1'):
    sys.path.append('/content/MLDL2024_project1')
try:
    from models.bisenet.build_bisenet import BiSeNet
except ImportError:
    print("Error: BiSeNet not found.")

# --- DICE LOSS ---
class DiceLoss(nn.Module):
    def __init__(self, num_classes=19, smooth=1.0, ignore_index=255):
        super(DiceLoss, self).__init__()
        self.num_classes = num_classes
        self.smooth = smooth
        self.ignore_index = ignore_index

    def forward(self, pred, target):
        pred = F.softmax(pred, dim=1)
        target_one_hot = F.one_hot(target.clamp(0, self.num_classes-1), num_classes=self.num_classes)
        target_one_hot = target_one_hot.permute(0, 3, 1, 2).float()

        mask = (target != self.ignore_index).unsqueeze(1).float()
        pred = pred * mask
        target_one_hot = target_one_hot * mask

        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))

        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return 1.0 - dice.mean()

# --- DATASET ---
class GTA5_City_Dataset(Dataset):
    def __init__(self, gta_root, city_root):
        self.gta_images_dir = os.path.join(gta_root, 'images')
        self.gta_masks_dir = os.path.join(gta_root, 'labels')
        self.gta_images = sorted(os.listdir(self.gta_images_dir))
        self.city_images_dir = os.path.join(city_root, 'leftImg8bit', 'train')
        self.city_images = []
        if os.path.exists(self.city_images_dir):
            for city in os.listdir(self.city_images_dir):
                c_path = os.path.join(self.city_images_dir, city)
                if os.path.isdir(c_path):
                    for f in os.listdir(c_path):
                        if f.endswith('_leftImg8bit.png'):
                            self.city_images.append(os.path.join(c_path, f))

        self.normalize = transforms.Compose([
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.id_mapping = {
            7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
            19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
            26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18
        }
        self.to_tensor = transforms.ToTensor()

    def __len__(self): return len(self.gta_images)
    def __getitem__(self, idx):
        gta_path = os.path.join(self.gta_images_dir, self.gta_images[idx])
        mask_path = os.path.join(self.gta_masks_dir, self.gta_images[idx])
        gta_img = Image.open(gta_path).convert('RGB').resize((1280, 720), Image.BILINEAR)
        gta_mask = Image.open(mask_path).resize((1280, 720), Image.NEAREST)

        rand_idx = random.randint(0, len(self.city_images) - 1)
        city_img = Image.open(self.city_images[rand_idx]).convert('RGB').resize((1280, 720), Image.BILINEAR)

        gta_t = self.normalize(self.to_tensor(gta_img))
        city_t = self.normalize(self.to_tensor(city_img))

        mask_np = np.array(gta_mask)
        gta_lbl = np.full(mask_np.shape, 255, dtype=np.uint8)
        for k, v in self.id_mapping.items(): gta_lbl[mask_np == k] = v

        return gta_t, torch.from_numpy(gta_lbl).long(), city_t

# --- TRAINING LOOP ---
print("Continuing Training (Epochs 16-30)...")
torch.cuda.empty_cache()

model = BiSeNet(num_classes=19, context_path='resnet18').to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

ce_criterion = nn.CrossEntropyLoss(ignore_index=255)
dice_criterion = DiceLoss(num_classes=19, ignore_index=255)

# Load the previous session
if os.path.exists(PRETRAINED_PATH):
    print(f"Loading previous best: {PRETRAINED_PATH}")
    checkpoint = torch.load(PRETRAINED_PATH, map_location=DEVICE)
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint)
else:
    print("Error: Previous model not found.")
    sys.exit()

dataset = GTA5_City_Dataset(GTA_PATH, CITYSCAPES_PATH)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)

model.train()
optimizer.zero_grad()

# Resume loop
start_epoch = 15

print(f"Training from Epoch {start_epoch} to {TOTAL_EPOCHS}...")

for epoch in range(start_epoch, TOTAL_EPOCHS):
    for i, (gta_img, gta_lbl, city_img) in enumerate(loader):
        gta_img, gta_lbl, city_img = gta_img.to(DEVICE), gta_lbl.to(DEVICE), city_img.to(DEVICE)

        # 1. Pseudo-Labeling
        with torch.no_grad():
            city_out = model(city_img)
            if isinstance(city_out, tuple): city_out = city_out[0]
            city_pseudo_lbl = torch.argmax(city_out, dim=1)

        # 2. Mix
        batch_size = gta_img.shape[0]
        mixed_img = city_img.clone()
        mixed_lbl = city_pseudo_lbl.clone()
        for b in range(batch_size):
            classes = torch.unique(gta_lbl[b])
            classes = classes[classes != 255]
            if len(classes) > 0:
                perm = torch.randperm(len(classes))
                sel = classes[perm[:(len(classes)+1)//2]]
                mask = torch.zeros_like(gta_lbl[b]).bool()
                for c in sel: mask = mask | (gta_lbl[b] == c)
                mixed_img[b, :, mask] = gta_img[b, :, mask]
                mixed_lbl[b, mask] = gta_lbl[b, mask]

        # 3. Forward
        out = model(mixed_img)

        # 4. Hybrid Loss
        loss_ce = ce_criterion(out[0], mixed_lbl)
        loss_dice = dice_criterion(out[0], mixed_lbl)

        total_loss = loss_ce + loss_dice
        total_loss += 0.1 * ce_criterion(out[1], mixed_lbl) + 0.1 * ce_criterion(out[2], mixed_lbl)

        total_loss = total_loss / ACCUM_STEPS
        total_loss.backward()

        if (i + 1) % ACCUM_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()

        if i % 100 == 0:
            print(f"Ep {epoch+1} | It {i} | Loss: {total_loss.item() * ACCUM_STEPS:.4f}")

    # Save to NEW file
    torch.save({'model_state_dict': model.state_dict(), 'epoch': epoch}, SAVE_PATH)
    print(f"Epoch {epoch+1} Saved to {SAVE_NAME}")

Continuing Training (Epochs 16-30)...
Loading previous best: /content/drive/MyDrive/semseg/bisenet_extension_final.pth
Training from Epoch 15 to 30...
Ep 16 | It 0 | Loss: 0.7495
Ep 16 | It 100 | Loss: 0.6704
Ep 16 | It 200 | Loss: 0.5989
Ep 16 | It 300 | Loss: 0.5979
Ep 16 | It 400 | Loss: 0.6397
Ep 16 | It 500 | Loss: 0.6072
Ep 16 | It 600 | Loss: 0.5751
Epoch 16 Saved to bisenet_extension_30ep.pth
Ep 17 | It 0 | Loss: 0.5805
Ep 17 | It 100 | Loss: 0.7117
Ep 17 | It 200 | Loss: 0.6065
Ep 17 | It 300 | Loss: 0.5594
Ep 17 | It 400 | Loss: 0.5609
Ep 17 | It 500 | Loss: 0.7333
Ep 17 | It 600 | Loss: 0.6689
Epoch 17 Saved to bisenet_extension_30ep.pth
Ep 18 | It 0 | Loss: 0.6963
Ep 18 | It 100 | Loss: 0.7050
Ep 18 | It 200 | Loss: 0.7120
Ep 18 | It 300 | Loss: 0.5380
Ep 18 | It 400 | Loss: 0.7319
Ep 18 | It 500 | Loss: 0.5989
Ep 18 | It 600 | Loss: 0.7979
Epoch 18 Saved to bisenet_extension_30ep.pth
Ep 19 | It 0 | Loss: 0.7695
Ep 19 | It 100 | Loss: 0.6412
Ep 19 | It 200 | Loss: 0.7053
Ep

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import os
import random
import sys

# --- CONFIGURATION ---
SAVE_NAME = 'bisenet_extension_50ep.pth'
# Load the 30-epoch model
PRETRAINED_PATH = '/content/drive/MyDrive/semseg/bisenet_extension_30ep.pth'
SAVE_PATH = f'/content/drive/MyDrive/semseg/{SAVE_NAME}'

TOTAL_EPOCHS = 50    # Going from 30 -> 50
BATCH_SIZE = 4
ACCUM_STEPS = 2
LR = 1e-3
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
GTA_PATH = '/content/dataset/project_data/gta5'
CITYSCAPES_PATH = '/content/dataset/project_data/cityscapes'

# --- IMPORTS ---
if os.path.exists('/content/MLDL2024_project1'):
    sys.path.append('/content/MLDL2024_project1')
try:
    from models.bisenet.build_bisenet import BiSeNet
except ImportError:
    print("Error: BiSeNet not found.")

# --- DICE LOSS ---
class DiceLoss(nn.Module):
    def __init__(self, num_classes=19, smooth=1.0, ignore_index=255):
        super(DiceLoss, self).__init__()
        self.num_classes = num_classes
        self.smooth = smooth
        self.ignore_index = ignore_index

    def forward(self, pred, target):
        pred = F.softmax(pred, dim=1)
        target_one_hot = F.one_hot(target.clamp(0, self.num_classes-1), num_classes=self.num_classes)
        target_one_hot = target_one_hot.permute(0, 3, 1, 2).float()

        mask = (target != self.ignore_index).unsqueeze(1).float()
        pred = pred * mask
        target_one_hot = target_one_hot * mask

        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))

        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return 1.0 - dice.mean()

# --- DATASET ---
class GTA5_City_Dataset(Dataset):
    def __init__(self, gta_root, city_root):
        self.gta_images_dir = os.path.join(gta_root, 'images')
        self.gta_masks_dir = os.path.join(gta_root, 'labels')
        self.gta_images = sorted(os.listdir(self.gta_images_dir))
        self.city_images_dir = os.path.join(city_root, 'leftImg8bit', 'train')
        self.city_images = []
        if os.path.exists(self.city_images_dir):
            for city in os.listdir(self.city_images_dir):
                c_path = os.path.join(self.city_images_dir, city)
                if os.path.isdir(c_path):
                    for f in os.listdir(c_path):
                        if f.endswith('_leftImg8bit.png'):
                            self.city_images.append(os.path.join(c_path, f))

        self.normalize = transforms.Compose([
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.id_mapping = {
            7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
            19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
            26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18
        }
        self.to_tensor = transforms.ToTensor()

    def __len__(self): return len(self.gta_images)
    def __getitem__(self, idx):
        gta_path = os.path.join(self.gta_images_dir, self.gta_images[idx])
        mask_path = os.path.join(self.gta_masks_dir, self.gta_images[idx])
        gta_img = Image.open(gta_path).convert('RGB').resize((1280, 720), Image.BILINEAR)
        gta_mask = Image.open(mask_path).resize((1280, 720), Image.NEAREST)

        rand_idx = random.randint(0, len(self.city_images) - 1)
        city_img = Image.open(self.city_images[rand_idx]).convert('RGB').resize((1280, 720), Image.BILINEAR)

        gta_t = self.normalize(self.to_tensor(gta_img))
        city_t = self.normalize(self.to_tensor(city_img))

        mask_np = np.array(gta_mask)
        gta_lbl = np.full(mask_np.shape, 255, dtype=np.uint8)
        for k, v in self.id_mapping.items(): gta_lbl[mask_np == k] = v

        return gta_t, torch.from_numpy(gta_lbl).long(), city_t

# --- TRAINING LOOP ---
print("Continuing Training (Epochs 31-50)...")
torch.cuda.empty_cache()

model = BiSeNet(num_classes=19, context_path='resnet18').to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

ce_criterion = nn.CrossEntropyLoss(ignore_index=255)
dice_criterion = DiceLoss(num_classes=19, ignore_index=255)

if os.path.exists(PRETRAINED_PATH):
    print(f"Loading previous best: {PRETRAINED_PATH}")
    checkpoint = torch.load(PRETRAINED_PATH, map_location=DEVICE)
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint)
else:
    print("Error: Previous model not found.")
    sys.exit()

dataset = GTA5_City_Dataset(GTA_PATH, CITYSCAPES_PATH)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)

model.train()
optimizer.zero_grad()

start_epoch = 30 # Resume from 30

print(f"Training from Epoch {start_epoch} to {TOTAL_EPOCHS}...")

for epoch in range(start_epoch, TOTAL_EPOCHS):
    for i, (gta_img, gta_lbl, city_img) in enumerate(loader):
        gta_img, gta_lbl, city_img = gta_img.to(DEVICE), gta_lbl.to(DEVICE), city_img.to(DEVICE)

        # 1. Pseudo-Labeling
        with torch.no_grad():
            city_out = model(city_img)
            if isinstance(city_out, tuple): city_out = city_out[0]
            city_pseudo_lbl = torch.argmax(city_out, dim=1)

        # 2. Mix
        batch_size = gta_img.shape[0]
        mixed_img = city_img.clone()
        mixed_lbl = city_pseudo_lbl.clone()
        for b in range(batch_size):
            classes = torch.unique(gta_lbl[b])
            classes = classes[classes != 255]
            if len(classes) > 0:
                perm = torch.randperm(len(classes))
                sel = classes[perm[:(len(classes)+1)//2]]
                mask = torch.zeros_like(gta_lbl[b]).bool()
                for c in sel: mask = mask | (gta_lbl[b] == c)
                mixed_img[b, :, mask] = gta_img[b, :, mask]
                mixed_lbl[b, mask] = gta_lbl[b, mask]

        # 3. Forward
        out = model(mixed_img)

        # 4. Hybrid Loss
        loss_ce = ce_criterion(out[0], mixed_lbl)
        loss_dice = dice_criterion(out[0], mixed_lbl)

        total_loss = loss_ce + loss_dice
        total_loss += 0.1 * ce_criterion(out[1], mixed_lbl) + 0.1 * ce_criterion(out[2], mixed_lbl)

        total_loss = total_loss / ACCUM_STEPS
        total_loss.backward()

        if (i + 1) % ACCUM_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()

        if i % 100 == 0:
            print(f"Ep {epoch+1} | It {i} | Loss: {total_loss.item() * ACCUM_STEPS:.4f}")

    torch.save({'model_state_dict': model.state_dict(), 'epoch': epoch}, SAVE_PATH)
    print(f"Epoch {epoch+1} Saved to {SAVE_NAME}")

Continuing Training (Epochs 31-50)...
Loading previous best: /content/drive/MyDrive/semseg/bisenet_extension_30ep.pth
Training from Epoch 30 to 50...
Ep 31 | It 0 | Loss: 0.7069
Ep 31 | It 100 | Loss: 0.6903
Ep 31 | It 200 | Loss: 0.5758
Ep 31 | It 300 | Loss: 0.6704
Ep 31 | It 400 | Loss: 0.6485
Ep 31 | It 500 | Loss: 0.6363
Ep 31 | It 600 | Loss: 0.6646
Epoch 31 Saved to bisenet_extension_50ep.pth
Ep 32 | It 0 | Loss: 0.6212
Ep 32 | It 100 | Loss: 0.6226
Ep 32 | It 200 | Loss: 0.7419
Ep 32 | It 300 | Loss: 0.5441
Ep 32 | It 400 | Loss: 0.5918
Ep 32 | It 500 | Loss: 0.5758
Ep 32 | It 600 | Loss: 0.6510
Epoch 32 Saved to bisenet_extension_50ep.pth
Ep 33 | It 0 | Loss: 0.5684
Ep 33 | It 100 | Loss: 0.6872
Ep 33 | It 200 | Loss: 0.6759
Ep 33 | It 300 | Loss: 0.5872
Ep 33 | It 400 | Loss: 0.5764
Ep 33 | It 500 | Loss: 0.6471
Ep 33 | It 600 | Loss: 0.5845
Epoch 33 Saved to bisenet_extension_50ep.pth
Ep 34 | It 0 | Loss: 0.5457
Ep 34 | It 100 | Loss: 0.5095
Ep 34 | It 200 | Loss: 0.4730
Ep 

In [ ]:
import torch
import numpy as np
import os
import sys
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms

# --- CONFIGURATION ---
# Point to the 50-epoch model
CHECKPOINT_PATH = '/content/drive/MyDrive/semseg/bisenet_extension_50ep.pth'
CITY_PATH = '/content/dataset/project_data/cityscapes'
NUM_CLASSES = 19
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- IMPORTS ---
if os.path.exists('/content/MLDL2024_project1'):
    sys.path.append('/content/MLDL2024_project1')
try:
    from models.bisenet.build_bisenet import BiSeNet
except:
    print("Warning: Could not import BiSeNet directly.")

# --- DATASET ---
class CityscapesValDataset(Dataset):
    def __init__(self, root):
        self.img_dir = os.path.join(root, 'leftImg8bit', 'val')
        self.lbl_dir = os.path.join(root, 'gtFine', 'val')
        self.imgs = []
        self.lbls = []
        if os.path.exists(self.img_dir):
            for city in sorted(os.listdir(self.img_dir)):
                img_path = os.path.join(self.img_dir, city)
                lbl_path = os.path.join(self.lbl_dir, city)
                for f in sorted(os.listdir(img_path)):
                    if f.endswith('_leftImg8bit.png'):
                        self.imgs.append(os.path.join(img_path, f))
                        self.lbls.append(os.path.join(lbl_path, f.replace('_leftImg8bit.png', '_gtFine_labelTrainIds.png')))
        self.transform = transforms.Compose([
            transforms.Resize((512, 1024)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    def __len__(self): return len(self.imgs)
    def __getitem__(self, idx):
        img = Image.open(self.imgs[idx]).convert('RGB')
        lbl = Image.open(self.lbls[idx]).resize((1024, 512), Image.NEAREST)
        img = self.transform(img)
        lbl = torch.from_numpy(np.array(lbl)).long()
        return img, lbl

# --- EVALUATION ---
print(f"Evaluating 50-Epoch Extension Model: {CHECKPOINT_PATH}")
model = BiSeNet(num_classes=NUM_CLASSES, context_path='resnet18').to(DEVICE)

if os.path.exists(CHECKPOINT_PATH):
    ckpt = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
    if isinstance(ckpt, dict) and 'model_state_dict' in ckpt:
        model.load_state_dict(ckpt['model_state_dict'])
    else:
        model.load_state_dict(ckpt)
    print("Model loaded successfully.")
else:
    print("Error: Checkpoint not found.")
    sys.exit()

model.eval()
loader = DataLoader(CityscapesValDataset(CITY_PATH), batch_size=1, shuffle=False, num_workers=2)
hist = np.zeros((NUM_CLASSES, NUM_CLASSES))

print("Processing images...")
with torch.no_grad():
    for img, lbl in tqdm(loader):
        img = img.to(DEVICE)
        output = model(img)
        if isinstance(output, tuple): output = output[0]
        preds = torch.argmax(output, dim=1).cpu().numpy()
        lbl = lbl.numpy()

        mask = (lbl >= 0) & (lbl < NUM_CLASSES)
        hist += np.bincount(
            NUM_CLASSES * lbl[mask].astype(int) + preds[mask],
            minlength=NUM_CLASSES ** 2
        ).reshape(NUM_CLASSES, NUM_CLASSES)

iou = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
miou = np.nanmean(iou) * 100

print("\n" + "="*30)
print(f"FINAL SCORE (Hybrid Loss - 50 Epochs): {miou:.2f}%")
print("="*30)

CLASSES = ["Road", "Sidewalk", "Building", "Wall", "Fence", "Pole", "Traffic Light", "Traffic Sign", "Vegetation", "Terrain", "Sky", "Person", "Rider", "Car", "Truck", "Bus", "Train", "Motorcycle", "Bicycle"]
for i, name in enumerate(CLASSES):
    print(f"{name:15s}: {iou[i]*100:.2f}%")

Evaluating 50-Epoch Extension Model: /content/drive/MyDrive/semseg/bisenet_extension_50ep.pth
Model loaded successfully.
Processing images...


100%|██████████| 500/500 [01:08<00:00,  7.34it/s]


FINAL SCORE (Hybrid Loss - 50 Epochs): 28.23%
Road           : 88.75%
Sidewalk       : 33.53%
Building       : 70.65%
Wall           : 20.21%
Fence          : 16.08%
Pole           : 21.74%
Traffic Light  : 20.19%
Traffic Sign   : 10.34%
Vegetation     : 71.40%
Terrain        : 13.73%
Sky            : 49.09%
Person         : 37.50%
Rider          : 0.00%
Car            : 71.37%
Truck          : 10.69%
Bus            : 1.02%
Train          : 0.00%
Motorcycle     : 0.00%
Bicycle        : 0.00%
